# CatBoost Starter for Brain Comp
EEGの差分の集計特徴量を用いたNotebook。

- ver1: CV = 1.26,
- ver2: CV = 1.175, LB = 0.93
- ver3: CV = 1.168, LB = 0.98


### Version Notes
- Version 1 - EEGのT=20, 30, 40秒における、10秒時間窓 / 20秒時間窓のmean, min, max, std, max-minを特徴量とした。また、データの1/20のみ利用。
- Version 2 - EEGのT=20, 30秒における、10秒時間窓 / 20秒時間窓のmean, min, max, std, max-minを特徴量とした。時間窓に必要な部分のみ計算することで高速化した。
- Version 3 - EEGのT=10, 20, 25, 30, 40秒における、5秒時間窓 / 10秒時間窓のmean, min, max, std, max-minを特徴量とした。

# Load Libraries

In [39]:
import os, gc
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
import pandas as pd
import numpy as np
import polars as pl
import matplotlib.pyplot as plt
import time
pl.Config.set_tbl_cols(-1)

VER = 3

# Load Test Data

In [40]:
df_test = pl.read_csv('../input/hms-harmful-brain-activity-classification/test.csv')
print('Train shape:', df_test.shape )
df_test.head()

Train shape: (1, 3)


spectrogram_id,eeg_id,patient_id
i64,i64,i64
853520,3911565283,6885


# Create Multiple Eeg Id Train Data
データの説明[here][1]には、テストデータには、同じ `eeg_id` からの複数の crop は含まれていないと記載されている。

[1]: https://www.kaggle.com/competitions/hms-harmful-brain-activity-classification/discussion/467021

# Feature Engineer


CatBoostモデルに入れる特徴量を作成する。


In [41]:
# parameter
# 差分特徴量の集計特徴量をつくる
# window = 4000 # 20秒窓 (20秒 / (1レコード1/200秒) = 4000レコード)
center_seconds_list = [10, 10, 20, 20, 25, 25, 30, 30, 40, 40]
window_list         = [1000, 2000, 1000, 2000, 1000, 2000, 1000, 2000, 1000, 2000]

In [42]:
def create_aggregated_features_on_EEG_differences(df_eeg_diff, window, center_sec):
    """
    @Args
    df_eeg_diff : 差分特徴量を持つデータフレーム
    window      : 時間窓として抽出する行数 [行]
    center_sec  : 時間窓の中心時刻 [s]
    
    @Returns
    df_features    : 集計特徴量を持つデータフレーム
    """
    ### 時間窓の設定
    window_time = window / (200)
    str_window = str(int(window_time)) + "s"

    # 時間窓 [window_min, window_max] に入るレコードのみ抽出
    window_max = center_sec + (window / 2)
    window_min = center_sec - (window / 2)

    df_eeg_diff = df_eeg_diff.filter((pl.col("seconds_eeg") <= window_max) & (pl.col("seconds_eeg") >= window_min))
    df_eeg_diff = df_eeg_diff.drop(['seconds_eeg'])

    
    ### 特徴量作成
    
    df_features = df_eeg_diff.group_by(by = "eeg_id").agg(
        ### 簡易特徴量
        # LL: Fp1 - T3
        pl.col("LL_Fp1-T3").mean().alias("mean_" + str_window + "_LL_Fp1-T3"),
        pl.col("LL_Fp1-T3").max().alias("min_" + str_window + "_LL_Fp1-T3"),
        pl.col("LL_Fp1-T3").min().alias("max_" + str_window + "_LL_Fp1-T3"),
        pl.col("LL_Fp1-T3").std().alias("std_" + str_window + "_LL_Fp1-T3"),
        (pl.col("LL_Fp1-T3").max() - pl.col("LL_Fp1-T3").min()).alias("max-min_" + str_window + "_LL_Fp1-T3"),

        # LL: Fp1 - T3
        pl.col("LL_T3-O1").mean().alias("mean_" + str_window + "_LL_T3-O1"),
        pl.col("LL_T3-O1").max().alias("min_" + str_window + "_LL_T3-O1"),
        pl.col("LL_T3-O1").min().alias("max_" + str_window + "_LL_T3-O1"),
        pl.col("LL_T3-O1").std().alias("std_" + str_window + "_LL_T3-O1"),
        (pl.col("LL_T3-O1").max() - pl.col("LL_T3-O1").min()).alias("max-min__" + str_window + "_LL_T3-O1"),


        # LP: Fp1 - C3
        pl.col("LP_Fp1-C3").mean().alias("mean_" + str_window + "_LP_Fp1-C3"),
        pl.col("LP_Fp1-C3").max().alias("min_" + str_window + "_LP_Fp1-C3"),
        pl.col("LP_Fp1-C3").min().alias("max_" + str_window + "_LP_Fp1-C3"),
        pl.col("LP_Fp1-C3").std().alias("std_" + str_window + "_LP_Fp1-C3"),
        (pl.col("LP_Fp1-C3").max() - pl.col("LP_Fp1-C3").min()).alias("max-min_" + str_window + "_LP_Fp1-C3"),

        # LP: C3 - O1
        pl.col("LP_C3-O1").mean().alias("mean_" + str_window + "_LP_C3-O1"),
        pl.col("LP_C3-O1").max().alias("min_" + str_window + "_LP_C3-O1"),
        pl.col("LP_C3-O1").min().alias("max_" + str_window + "_LP_C3-O1"),
        pl.col("LP_C3-O1").std().alias("std_" + str_window + "_LP_C3-O1"),
        (pl.col("LP_C3-O1").max() - pl.col("LP_C3-O1").min()).alias("max-min_" + str_window + "_LP_C3-O1"),


        # RP: Fp1 - C4
        pl.col("RP_Fp2-C4").mean().alias("mean_" + str_window + "_RP_Fp2-C4"),
        pl.col("RP_Fp2-C4").max().alias("min_" + str_window + "_RP_Fp2-C4"),
        pl.col("RP_Fp2-C4").min().alias("max_" + str_window + "_RP_Fp2-C4"),
        pl.col("RP_Fp2-C4").std().alias("std_" + str_window + "_RP_Fp2-C4"),
        (pl.col("RP_Fp2-C4").max() - pl.col("RP_Fp2-C4").min()).alias("max-min_" + str_window + "_RP_Fp2-C4"),

        # RP: C4 - O2
        pl.col("RP_C4-O2").mean().alias("mean_" + str_window + "_RP_C4-O2"),
        pl.col("RP_C4-O2").max().alias("min_" + str_window + "_RP_C4-O2"),
        pl.col("RP_C4-O2").min().alias("max_" + str_window + "_RP_C4-O2"),
        pl.col("RP_C4-O2").std().alias("std_" + str_window + "_RP_C4-O2"),
        (pl.col("RP_C4-O2").max() - pl.col("RP_C4-O2").min()).alias("max-min_" + str_window + "_RP_C4-O2"),


        # RR: Fp2 - T4
        pl.col("RR_Fp2-T4").mean().alias("mean_" + str_window + "_RR_Fp2-T4"),
        pl.col("RR_Fp2-T4").max().alias("min_" + str_window + "_RR_Fp2-T4"),
        pl.col("RR_Fp2-T4").min().alias("max_" + str_window + "_RR_Fp2-T4"),
        pl.col("RR_Fp2-T4").std().alias("std_" + str_window + "_RR_Fp2-T4"),
        (pl.col("RR_Fp2-T4").max() - pl.col("RR_Fp2-T4").min()).alias("max-min_" + str_window + "_RR_Fp2-T4"),

        # RR: T4 - O2
        pl.col("RR_T4-O2").mean().alias("mean_" + str_window + "_RR_T4-O2"),
        pl.col("RR_T4-O2").max().alias("min_" + str_window + "_RR_T4-O2"),
        pl.col("RR_T4-O2").min().alias("max_" + str_window + "_RR_T4-O2"),
        pl.col("RR_T4-O2").std().alias("std_" + str_window + "_RR_T4-O2"),
        (pl.col("RR_T4-O2").max() - pl.col("RR_T4-O2").min()).alias("max-min_" + str_window + "_RR_T4-O2"),

        
        ### 全差分の特徴量
        # LL: Fp1 - F7
        pl.col("LL_Fp1-F7").mean().alias("mean_" + str_window + "_LL_Fp1-F7"),
        pl.col("LL_Fp1-F7").max().alias("min_" + str_window + "_LL_Fp1-F7"),
        pl.col("LL_Fp1-F7").min().alias("max_" + str_window + "_LL_Fp1-F7"),
        pl.col("LL_Fp1-F7").std().alias("std_" + str_window + "_LL_Fp1-F7"),
        (pl.col("LL_Fp1-F7").max() - pl.col("LL_Fp1-F7").min()).alias("max-min_" + str_window + "_LL_Fp1-F7"),

        # LL: F7 - T3
        pl.col("LL_F7-T3").mean().alias("mean_" + str_window + "_LL_F7-T3"),
        pl.col("LL_F7-T3").max().alias("min_" + str_window + "_LL_F7-T3"),
        pl.col("LL_F7-T3").min().alias("max_" + str_window + "_LL_F7-T3"),
        pl.col("LL_F7-T3").std().alias("std_" + str_window + "_LL_F7-T3"),
        (pl.col("LL_F7-T3").max() - pl.col("LL_F7-T3").min()).alias("max-min_" + str_window + "_LL_F7-T3"),

        # LL: T3 - T5
        pl.col("LL_T3-T5").mean().alias("mean_" + str_window + "_LL_T3-T5"),
        pl.col("LL_T3-T5").max().alias("min_" + str_window + "_LL_T3-T5"),
        pl.col("LL_T3-T5").min().alias("max_" + str_window + "_LL_T3-T5"),
        pl.col("LL_T3-T5").std().alias("std_" + str_window + "_LL_T3-T5"),
        (pl.col("LL_T3-T5").max() - pl.col("LL_T3-T5").min()).alias("max-min_" + str_window + "_LL_T3-T5"),

        # LL: T5 - O1
        pl.col("LL_T5-O1").mean().alias("mean_" + str_window + "_LL_T5-O1"),
        pl.col("LL_T5-O1").max().alias("min_" + str_window + "_LL_T5-O1"),
        pl.col("LL_T5-O1").min().alias("max_" + str_window + "_LL_T5-O1"),
        pl.col("LL_T5-O1").std().alias("std_" + str_window + "_LL_T5-O1"),
        (pl.col("LL_T5-O1").max() - pl.col("LL_T5-O1").min()).alias("max-min_" + str_window + "_LL_T5-O1"),



        # LP: Fp1 - F3
        pl.col("LP_Fp1-F3").mean().alias("mean_" + str_window + "_LP_Fp1-F3"),
        pl.col("LP_Fp1-F3").max().alias("min_" + str_window + "_LP_Fp1-F3"),
        pl.col("LP_Fp1-F3").min().alias("max_" + str_window + "_LP_Fp1-F3"),
        pl.col("LP_Fp1-F3").std().alias("std_" + str_window + "_LP_Fp1-F3"),
        (pl.col("LP_Fp1-F3").max() - pl.col("LP_Fp1-F3").min()).alias("max-min_" + str_window + "_LP_Fp1-F3"),

        # LP: F3 - C3
        pl.col("LP_F3-C3").mean().alias("mean_" + str_window + "_LP_F3-C3"),
        pl.col("LP_F3-C3").max().alias("min_" + str_window + "_LP_F3-C3"),
        pl.col("LP_F3-C3").min().alias("max_" + str_window + "_LP_F3-C3"),
        pl.col("LP_F3-C3").std().alias("std_" + str_window + "_LP_F3-C3"),
        (pl.col("LP_F3-C3").max() - pl.col("LP_F3-C3").min()).alias("max-min_" + str_window + "_LP_F3-C3"),

        # LP: C3 - P3
        pl.col("LP_C3-P3").mean().alias("mean_" + str_window + "_LP_C3-P3"),
        pl.col("LP_C3-P3").max().alias("min_" + str_window + "_LP_C3-P3"),
        pl.col("LP_C3-P3").min().alias("max_" + str_window + "_LP_C3-P3"),
        pl.col("LP_C3-P3").std().alias("std_" + str_window + "_LP_C3-P3"),
        (pl.col("LP_C3-P3").max() - pl.col("LP_C3-P3").min()).alias("max-min_" + str_window + "_LP_C3-P3"),

        # LP: P3 - O1
        pl.col("LP_P3-O1").mean().alias("mean_" + str_window + "_LP_P3-O1"),
        pl.col("LP_P3-O1").max().alias("min_" + str_window + "_LP_P3-O1"),
        pl.col("LP_P3-O1").min().alias("max_" + str_window + "_LP_P3-O1"),
        pl.col("LP_P3-O1").std().alias("std_" + str_window + "_LP_P3-O1"),
        (pl.col("LP_P3-O1").max() - pl.col("LP_P3-O1").min()).alias("max-min_" + str_window + "_LP_P3-O1"),



        # RP: Fp2 - F4
        pl.col("RP_Fp2-F4").mean().alias("mean_" + str_window + "_RP_Fp2-F4"),
        pl.col("RP_Fp2-F4").max().alias("min_" + str_window + "_RP_Fp2-F4"),
        pl.col("RP_Fp2-F4").min().alias("max_" + str_window + "_RP_Fp2-F4"),
        pl.col("RP_Fp2-F4").std().alias("std_" + str_window + "_RP_Fp2-F4"),
        (pl.col("RP_Fp2-F4").max() - pl.col("RP_Fp2-F4").min()).alias("max-min_" + str_window + "_RP_Fp2-F4"),

        # RP: F4 - C4
        pl.col("RP_F4-C4").mean().alias("mean_" + str_window + "_RP_F4-C4"),
        pl.col("RP_F4-C4").max().alias("min_" + str_window + "_RP_F4-C4"),
        pl.col("RP_F4-C4").min().alias("max_" + str_window + "_RP_F4-C4"),
        pl.col("RP_F4-C4").std().alias("std_" + str_window + "_RP_F4-C4"),
        (pl.col("RP_F4-C4").max() - pl.col("RP_F4-C4").min()).alias("max-min_" + str_window + "_RP_F4-C4"),

        # RP: C4 - P4
        pl.col("RP_C4-P4").mean().alias("mean_" + str_window + "_RP_C4-P4"),
        pl.col("RP_C4-P4").max().alias("min_" + str_window + "_RP_C4-P4"),
        pl.col("RP_C4-P4").min().alias("max_" + str_window + "_RP_C4-P4"),
        pl.col("RP_C4-P4").std().alias("std_" + str_window + "_RP_C4-P4"),
        (pl.col("RP_C4-P4").max() - pl.col("RP_C4-P4").min()).alias("max-min_" + str_window + "_RP_C4-P4"),

        # RP: P4 - O2
        pl.col("RP_P4-O2").mean().alias("mean_" + str_window + "_RP_P4-O2"),
        pl.col("RP_P4-O2").max().alias("min_" + str_window + "_RP_P4-O2"),
        pl.col("RP_P4-O2").min().alias("max_" + str_window + "_RP_P4-O2"),
        pl.col("RP_P4-O2").std().alias("std_" + str_window + "_RP_P4-O2"),
        (pl.col("RP_P4-O2").max() - pl.col("RP_P4-O2").min()).alias("max-min_" + str_window + "_RP_P4-O2"),


        
        # RR: Fp2 - F8
        pl.col("RR_Fp2-F8").mean().alias("mean_" + str_window + "_RR_Fp2-F8"),
        pl.col("RR_Fp2-F8").max().alias("min_" + str_window + "_RR_Fp2-F8"),
        pl.col("RR_Fp2-F8").min().alias("max_" + str_window + "_RR_Fp2-F8"),
        pl.col("RR_Fp2-F8").std().alias("std_" + str_window + "_RR_Fp2-F8"),
        (pl.col("RR_Fp2-F8").max() - pl.col("RR_Fp2-F8").min()).alias("max-min_" + str_window + "_RR_Fp2-F8"),

        # RR: F8 - T4
        pl.col("RR_F8-T4").mean().alias("mean_" + str_window + "_RR_F8-T4"),
        pl.col("RR_F8-T4").max().alias("min_" + str_window + "_RR_F8-T4"),
        pl.col("RR_F8-T4").min().alias("max_" + str_window + "_RR_F8-T4"),
        pl.col("RR_F8-T4").std().alias("std_" + str_window + "_RR_F8-T4"),
        (pl.col("RR_F8-T4").max() - pl.col("RR_F8-T4").min()).alias("max-min_" + str_window + "_RR_F8-T4"),

        # RR: T4 - T6
        pl.col("RR_T4-T6").mean().alias("mean_" + str_window + "_RR_T4-T6"),
        pl.col("RR_T4-T6").max().alias("min_" + str_window + "_RR_T4-T6"),
        pl.col("RR_T4-T6").min().alias("max_" + str_window + "_RR_T4-T6"),
        pl.col("RR_T4-T6").std().alias("std_" + str_window + "_RR_T4-T6"),
        (pl.col("RR_T4-T6").max() - pl.col("RR_T4-T6").min()).alias("max-min_" + str_window + "_RR_T4-T6"),

        # RR: T6 - O2
        pl.col("RR_T6-O2").mean().alias("mean_" + str_window + "_RR_T6-O2"),
        pl.col("RR_T6-O2").max().alias("min_" + str_window + "_RR_T6-O2"),
        pl.col("RR_T6-O2").min().alias("max_" + str_window + "_RR_T6-O2"),
        pl.col("RR_T6-O2").std().alias("std_" + str_window + "_RR_T6-O2"),
        (pl.col("RR_T6-O2").max() - pl.col("RR_T6-O2").min()).alias("max-min_" + str_window + "_RR_T6-O2"),        
    )

    
    # 差分特徴量をdrop
    df_features = df_features.drop(
        ['LL_Fp1-T3', 'LL_T3-O1', 'LP_Fp1-C3', 'LP_C3-O1', 'RP_Fp2-C4', 'RP_C4-O2', 'RR_Fp2-T4', 'RR_T4-O2',
         'LL_Fp1-F7', 'LL_F7-T3', 'LL_T3-T5', 'LL_T5-O1',
         'LP_Fp1-F3', 'LP_F3-C3', 'LP_C3-P3', 'LP_P3-O1',
         'RP_Fp2-F4', 'RP_F4-C4', 'RP_C4-P4', 'RP_P4-O2',
         'RR_Fp2-F8', 'RR_F8-T4', 'RR_T4-T6', 'RR_T6-O2',]
        )
    
    
    # 列名を一括で変更
    col_suffix = "_at_" + str(center_sec) + "s"
    list_original_col = df_features.columns
    df_features = df_features.with_columns(pl.all().name.suffix(col_suffix)).drop(list_original_col) # suffixを付与して、元の列名の列を削除
    
    # eeg_id 列の列名を戻す
    eeg_col = "eeg_id_at_" + str(center_sec) + "s"
    df_features = df_features.rename({eeg_col : "eeg_id"})


    return df_features

In [43]:
%%time
PATH = '../input/hms-harmful-brain-activity-classification/test_eegs/'
files = os.listdir(PATH)
print(f'There are {len(files)} eeg parquet files')

start = time.time()
df_all_diff_features = pl.DataFrame()
for i, f in enumerate(files):
    if i%100 == 0:
        print(i, ', ', end = '')
        
        ### 状況確認
        # 時間計測
        end = time.time()
        seconds = end - start
        seconds = int(seconds + 0.5)    # 秒数を四捨五入
        h = seconds // 3600             # 時の取得
        m = (seconds - h * 3600) // 60  # 分の取得
        s = seconds - h * 3600 - m * 60 # 秒の取得
        print(f"{h:02}:{m:02}:{s:02}" )
        start = time.time()

        # 形状確認
        print(df_all_diff_features.shape)

    # if i%10 != 0: continue

    eeg_id = int(f.split('.')[0])    

    # 読み込み
    temp_eeg = pl.read_parquet(f'{PATH}{f}')

    # eeg_id を追加
    temp_eeg = temp_eeg.with_columns(pl.lit(str(eeg_id)).alias("eeg_id"))

    # eegデータにおける、計測開始から各行までの秒数 seconds_eeg
    t = 1/200 # EEGのサンプリング周波数は200Hzなので、1行は1/200秒ごとの計測値を表す
    temp_eeg = temp_eeg.with_row_count("id")
    temp_eeg = temp_eeg.with_columns((pl.col("id") * t).alias("seconds_eeg"))   # 計測時刻の列をつくる
    temp_eeg = temp_eeg.filter(pl.col("seconds_eeg") < 50).drop(["id"])         # 50秒以降のデータは使わない
    
    # 差分特徴量を作る
    temp_eeg = temp_eeg.with_columns(
        # 簡易特徴量
        (pl.col("Fp1") - pl.col("T3")).alias("LL_Fp1-T3"),
        (pl.col("T3") - pl.col("O1")).alias("LL_T3-O1"),
        (pl.col("Fp1") - pl.col("C3")).alias("LP_Fp1-C3"),
        (pl.col("C3") - pl.col("O1")).alias("LP_C3-O1"),
        (pl.col("Fp2") - pl.col("C4")).alias("RP_Fp2-C4"),
        (pl.col("C4") - pl.col("O2")).alias("RP_C4-O2"),
        (pl.col("Fp2") - pl.col("T4")).alias("RR_Fp2-T4"),
        (pl.col("T4") - pl.col("O2")).alias("RR_T4-O2"),

        # 全差分の特徴量
        (pl.col("Fp1") - pl.col("F7")).alias("LL_Fp1-F7"),
        (pl.col("F7") - pl.col("T3")).alias("LL_F7-T3"),
        (pl.col("T3") - pl.col("T5")).alias("LL_T3-T5"),
        (pl.col("T5") - pl.col("O1")).alias("LL_T5-O1"),

        (pl.col("Fp1") - pl.col("F3")).alias("LP_Fp1-F3"),
        (pl.col("F3") - pl.col("C3")).alias("LP_F3-C3"),
        (pl.col("C3") - pl.col("P3")).alias("LP_C3-P3"),
        (pl.col("P3") - pl.col("O1")).alias("LP_P3-O1"),

        (pl.col("Fp2") - pl.col("F4")).alias("RP_Fp2-F4"),
        (pl.col("F4") - pl.col("C4")).alias("RP_F4-C4"),
        (pl.col("C4") - pl.col("P4")).alias("RP_C4-P4"),
        (pl.col("P4") - pl.col("O2")).alias("RP_P4-O2"),

        (pl.col("Fp2") - pl.col("F8")).alias("RR_Fp2-F8"),
        (pl.col("F8") - pl.col("T4")).alias("RR_F8-T4"),
        (pl.col("T4") - pl.col("T6")).alias("RR_T4-T6"),
        (pl.col("T6") - pl.col("O2")).alias("RR_T6-O2"),
        
    )

    # 差分特徴量以外drop
    temp_eeg = temp_eeg.drop(['Fp1', 'F3', 'C3', 'P3', 'F7', 'T3', 'T5', 'O1', 'Fz', 'Cz', 'Pz', 'Fp2', 'F4', 'C4', 'P4', 'F8', 'T4', 'T6', 'O2', 'EKG'])
    
    # eeg_id のみ持つデータフレームを作成
    df_eeg_feature = pl.DataFrame({"eeg_id": str(eeg_id)})

    # ひとつのeeg_id のデータフレームに対して特徴量を作成
    for center_sec, window in zip(center_seconds_list, window_list):
            df_features = create_aggregated_features_on_EEG_differences(temp_eeg, window, center_sec)
            df_eeg_feature = df_eeg_feature.join(df_features, on = "eeg_id", how = "left")

    # 全ての eeg_id に対して結合
    df_all_diff_features = pl.concat([df_all_diff_features, df_eeg_feature], how = 'vertical')

There are 1 eeg parquet files
0 , 00:00:00
(0, 0)
CPU times: total: 0 ns
Wall time: 7.35 ms


In [44]:
df_all_diff_features.head()

eeg_id,mean_5s_LL_Fp1-T3_at_20s,min_5s_LL_Fp1-T3_at_20s,max_5s_LL_Fp1-T3_at_20s,std_5s_LL_Fp1-T3_at_20s,max-min_5s_LL_Fp1-T3_at_20s,mean_5s_LL_T3-O1_at_20s,min_5s_LL_T3-O1_at_20s,max_5s_LL_T3-O1_at_20s,std_5s_LL_T3-O1_at_20s,max-min__5s_LL_T3-O1_at_20s,mean_5s_LP_Fp1-C3_at_20s,min_5s_LP_Fp1-C3_at_20s,max_5s_LP_Fp1-C3_at_20s,std_5s_LP_Fp1-C3_at_20s,max-min_5s_LP_Fp1-C3_at_20s,mean_5s_LP_C3-O1_at_20s,min_5s_LP_C3-O1_at_20s,max_5s_LP_C3-O1_at_20s,std_5s_LP_C3-O1_at_20s,max-min_5s_LP_C3-O1_at_20s,mean_5s_RP_Fp2-C4_at_20s,min_5s_RP_Fp2-C4_at_20s,max_5s_RP_Fp2-C4_at_20s,std_5s_RP_Fp2-C4_at_20s,max-min_5s_RP_Fp2-C4_at_20s,mean_5s_RP_C4-O2_at_20s,min_5s_RP_C4-O2_at_20s,max_5s_RP_C4-O2_at_20s,std_5s_RP_C4-O2_at_20s,max-min_5s_RP_C4-O2_at_20s,mean_5s_RR_Fp2-T4_at_20s,min_5s_RR_Fp2-T4_at_20s,max_5s_RR_Fp2-T4_at_20s,std_5s_RR_Fp2-T4_at_20s,max-min_5s_RR_Fp2-T4_at_20s,mean_5s_RR_T4-O2_at_20s,min_5s_RR_T4-O2_at_20s,max_5s_RR_T4-O2_at_20s,std_5s_RR_T4-O2_at_20s,max-min_5s_RR_T4-O2_at_20s,mean_10s_LL_Fp1-T3_at_20s,min_10s_LL_Fp1-T3_at_20s,max_10s_LL_Fp1-T3_at_20s,std_10s_LL_Fp1-T3_at_20s,max-min_10s_LL_Fp1-T3_at_20s,mean_10s_LL_T3-O1_at_20s,min_10s_LL_T3-O1_at_20s,max_10s_LL_T3-O1_at_20s,std_10s_LL_T3-O1_at_20s,max-min__10s_LL_T3-O1_at_20s,mean_10s_LP_Fp1-C3_at_20s,min_10s_LP_Fp1-C3_at_20s,max_10s_LP_Fp1-C3_at_20s,std_10s_LP_Fp1-C3_at_20s,max-min_10s_LP_Fp1-C3_at_20s,mean_10s_LP_C3-O1_at_20s,min_10s_LP_C3-O1_at_20s,max_10s_LP_C3-O1_at_20s,std_10s_LP_C3-O1_at_20s,max-min_10s_LP_C3-O1_at_20s,mean_10s_RP_Fp2-C4_at_20s,min_10s_RP_Fp2-C4_at_20s,max_10s_RP_Fp2-C4_at_20s,std_10s_RP_Fp2-C4_at_20s,max-min_10s_RP_Fp2-C4_at_20s,mean_10s_RP_C4-O2_at_20s,min_10s_RP_C4-O2_at_20s,max_10s_RP_C4-O2_at_20s,std_10s_RP_C4-O2_at_20s,max-min_10s_RP_C4-O2_at_20s,mean_10s_RR_Fp2-T4_at_20s,min_10s_RR_Fp2-T4_at_20s,max_10s_RR_Fp2-T4_at_20s,std_10s_RR_Fp2-T4_at_20s,max-min_10s_RR_Fp2-T4_at_20s,mean_10s_RR_T4-O2_at_20s,min_10s_RR_T4-O2_at_20s,max_10s_RR_T4-O2_at_20s,std_10s_RR_T4-O2_at_20s,max-min_10s_RR_T4-O2_at_20s,mean_5s_LL_Fp1-T3_at_30s,min_5s_LL_Fp1-T3_at_30s,max_5s_LL_Fp1-T3_at_30s,std_5s_LL_Fp1-T3_at_30s,max-min_5s_LL_Fp1-T3_at_30s,mean_5s_LL_T3-O1_at_30s,min_5s_LL_T3-O1_at_30s,max_5s_LL_T3-O1_at_30s,std_5s_LL_T3-O1_at_30s,max-min__5s_LL_T3-O1_at_30s,mean_5s_LP_Fp1-C3_at_30s,min_5s_LP_Fp1-C3_at_30s,max_5s_LP_Fp1-C3_at_30s,std_5s_LP_Fp1-C3_at_30s,max-min_5s_LP_Fp1-C3_at_30s,mean_5s_LP_C3-O1_at_30s,min_5s_LP_C3-O1_at_30s,max_5s_LP_C3-O1_at_30s,std_5s_LP_C3-O1_at_30s,max-min_5s_LP_C3-O1_at_30s,mean_5s_RP_Fp2-C4_at_30s,min_5s_RP_Fp2-C4_at_30s,max_5s_RP_Fp2-C4_at_30s,std_5s_RP_Fp2-C4_at_30s,max-min_5s_RP_Fp2-C4_at_30s,mean_5s_RP_C4-O2_at_30s,min_5s_RP_C4-O2_at_30s,max_5s_RP_C4-O2_at_30s,std_5s_RP_C4-O2_at_30s,max-min_5s_RP_C4-O2_at_30s,mean_5s_RR_Fp2-T4_at_30s,min_5s_RR_Fp2-T4_at_30s,max_5s_RR_Fp2-T4_at_30s,std_5s_RR_Fp2-T4_at_30s,max-min_5s_RR_Fp2-T4_at_30s,mean_5s_RR_T4-O2_at_30s,min_5s_RR_T4-O2_at_30s,max_5s_RR_T4-O2_at_30s,std_5s_RR_T4-O2_at_30s,max-min_5s_RR_T4-O2_at_30s,mean_10s_LL_Fp1-T3_at_30s,min_10s_LL_Fp1-T3_at_30s,max_10s_LL_Fp1-T3_at_30s,std_10s_LL_Fp1-T3_at_30s,max-min_10s_LL_Fp1-T3_at_30s,mean_10s_LL_T3-O1_at_30s,min_10s_LL_T3-O1_at_30s,max_10s_LL_T3-O1_at_30s,std_10s_LL_T3-O1_at_30s,max-min__10s_LL_T3-O1_at_30s,mean_10s_LP_Fp1-C3_at_30s,min_10s_LP_Fp1-C3_at_30s,max_10s_LP_Fp1-C3_at_30s,std_10s_LP_Fp1-C3_at_30s,max-min_10s_LP_Fp1-C3_at_30s,mean_10s_LP_C3-O1_at_30s,min_10s_LP_C3-O1_at_30s,max_10s_LP_C3-O1_at_30s,std_10s_LP_C3-O1_at_30s,max-min_10s_LP_C3-O1_at_30s,mean_10s_RP_Fp2-C4_at_30s,min_10s_RP_Fp2-C4_at_30s,max_10s_RP_Fp2-C4_at_30s,std_10s_RP_Fp2-C4_at_30s,max-min_10s_RP_Fp2-C4_at_30s,mean_10s_RP_C4-O2_at_30s,min_10s_RP_C4-O2_at_30s,max_10s_RP_C4-O2_at_30s,std_10s_RP_C4-O2_at_30s,max-min_10s_RP_C4-O2_at_30s,mean_10s_RR_Fp2-T4_at_30s,min_10s_RR_Fp2-T4_at_30s,max_10s_RR_Fp2-T4_at_30s,std_10s_RR_Fp2-T4_at_30s,max-min_10s_RR_Fp2-T4_at_30s,mean_10s_RR_T4-O2_at_30s,min_10s_RR_T4-O2_at_30s,max_10s_RR_T4-O2_at_30s,std_10s_RR_T4-O2_at_30s,max-min_10s_RR_T4

In [45]:
df_all_diff_features = df_all_diff_features.with_columns(pl.col("eeg_id").cast(pl.Int64).alias("eeg_id"))
df_all_diff_features.tail(5)

eeg_id,mean_5s_LL_Fp1-T3_at_20s,min_5s_LL_Fp1-T3_at_20s,max_5s_LL_Fp1-T3_at_20s,std_5s_LL_Fp1-T3_at_20s,max-min_5s_LL_Fp1-T3_at_20s,mean_5s_LL_T3-O1_at_20s,min_5s_LL_T3-O1_at_20s,max_5s_LL_T3-O1_at_20s,std_5s_LL_T3-O1_at_20s,max-min__5s_LL_T3-O1_at_20s,mean_5s_LP_Fp1-C3_at_20s,min_5s_LP_Fp1-C3_at_20s,max_5s_LP_Fp1-C3_at_20s,std_5s_LP_Fp1-C3_at_20s,max-min_5s_LP_Fp1-C3_at_20s,mean_5s_LP_C3-O1_at_20s,min_5s_LP_C3-O1_at_20s,max_5s_LP_C3-O1_at_20s,std_5s_LP_C3-O1_at_20s,max-min_5s_LP_C3-O1_at_20s,mean_5s_RP_Fp2-C4_at_20s,min_5s_RP_Fp2-C4_at_20s,max_5s_RP_Fp2-C4_at_20s,std_5s_RP_Fp2-C4_at_20s,max-min_5s_RP_Fp2-C4_at_20s,mean_5s_RP_C4-O2_at_20s,min_5s_RP_C4-O2_at_20s,max_5s_RP_C4-O2_at_20s,std_5s_RP_C4-O2_at_20s,max-min_5s_RP_C4-O2_at_20s,mean_5s_RR_Fp2-T4_at_20s,min_5s_RR_Fp2-T4_at_20s,max_5s_RR_Fp2-T4_at_20s,std_5s_RR_Fp2-T4_at_20s,max-min_5s_RR_Fp2-T4_at_20s,mean_5s_RR_T4-O2_at_20s,min_5s_RR_T4-O2_at_20s,max_5s_RR_T4-O2_at_20s,std_5s_RR_T4-O2_at_20s,max-min_5s_RR_T4-O2_at_20s,mean_10s_LL_Fp1-T3_at_20s,min_10s_LL_Fp1-T3_at_20s,max_10s_LL_Fp1-T3_at_20s,std_10s_LL_Fp1-T3_at_20s,max-min_10s_LL_Fp1-T3_at_20s,mean_10s_LL_T3-O1_at_20s,min_10s_LL_T3-O1_at_20s,max_10s_LL_T3-O1_at_20s,std_10s_LL_T3-O1_at_20s,max-min__10s_LL_T3-O1_at_20s,mean_10s_LP_Fp1-C3_at_20s,min_10s_LP_Fp1-C3_at_20s,max_10s_LP_Fp1-C3_at_20s,std_10s_LP_Fp1-C3_at_20s,max-min_10s_LP_Fp1-C3_at_20s,mean_10s_LP_C3-O1_at_20s,min_10s_LP_C3-O1_at_20s,max_10s_LP_C3-O1_at_20s,std_10s_LP_C3-O1_at_20s,max-min_10s_LP_C3-O1_at_20s,mean_10s_RP_Fp2-C4_at_20s,min_10s_RP_Fp2-C4_at_20s,max_10s_RP_Fp2-C4_at_20s,std_10s_RP_Fp2-C4_at_20s,max-min_10s_RP_Fp2-C4_at_20s,mean_10s_RP_C4-O2_at_20s,min_10s_RP_C4-O2_at_20s,max_10s_RP_C4-O2_at_20s,std_10s_RP_C4-O2_at_20s,max-min_10s_RP_C4-O2_at_20s,mean_10s_RR_Fp2-T4_at_20s,min_10s_RR_Fp2-T4_at_20s,max_10s_RR_Fp2-T4_at_20s,std_10s_RR_Fp2-T4_at_20s,max-min_10s_RR_Fp2-T4_at_20s,mean_10s_RR_T4-O2_at_20s,min_10s_RR_T4-O2_at_20s,max_10s_RR_T4-O2_at_20s,std_10s_RR_T4-O2_at_20s,max-min_10s_RR_T4-O2_at_20s,mean_5s_LL_Fp1-T3_at_30s,min_5s_LL_Fp1-T3_at_30s,max_5s_LL_Fp1-T3_at_30s,std_5s_LL_Fp1-T3_at_30s,max-min_5s_LL_Fp1-T3_at_30s,mean_5s_LL_T3-O1_at_30s,min_5s_LL_T3-O1_at_30s,max_5s_LL_T3-O1_at_30s,std_5s_LL_T3-O1_at_30s,max-min__5s_LL_T3-O1_at_30s,mean_5s_LP_Fp1-C3_at_30s,min_5s_LP_Fp1-C3_at_30s,max_5s_LP_Fp1-C3_at_30s,std_5s_LP_Fp1-C3_at_30s,max-min_5s_LP_Fp1-C3_at_30s,mean_5s_LP_C3-O1_at_30s,min_5s_LP_C3-O1_at_30s,max_5s_LP_C3-O1_at_30s,std_5s_LP_C3-O1_at_30s,max-min_5s_LP_C3-O1_at_30s,mean_5s_RP_Fp2-C4_at_30s,min_5s_RP_Fp2-C4_at_30s,max_5s_RP_Fp2-C4_at_30s,std_5s_RP_Fp2-C4_at_30s,max-min_5s_RP_Fp2-C4_at_30s,mean_5s_RP_C4-O2_at_30s,min_5s_RP_C4-O2_at_30s,max_5s_RP_C4-O2_at_30s,std_5s_RP_C4-O2_at_30s,max-min_5s_RP_C4-O2_at_30s,mean_5s_RR_Fp2-T4_at_30s,min_5s_RR_Fp2-T4_at_30s,max_5s_RR_Fp2-T4_at_30s,std_5s_RR_Fp2-T4_at_30s,max-min_5s_RR_Fp2-T4_at_30s,mean_5s_RR_T4-O2_at_30s,min_5s_RR_T4-O2_at_30s,max_5s_RR_T4-O2_at_30s,std_5s_RR_T4-O2_at_30s,max-min_5s_RR_T4-O2_at_30s,mean_10s_LL_Fp1-T3_at_30s,min_10s_LL_Fp1-T3_at_30s,max_10s_LL_Fp1-T3_at_30s,std_10s_LL_Fp1-T3_at_30s,max-min_10s_LL_Fp1-T3_at_30s,mean_10s_LL_T3-O1_at_30s,min_10s_LL_T3-O1_at_30s,max_10s_LL_T3-O1_at_30s,std_10s_LL_T3-O1_at_30s,max-min__10s_LL_T3-O1_at_30s,mean_10s_LP_Fp1-C3_at_30s,min_10s_LP_Fp1-C3_at_30s,max_10s_LP_Fp1-C3_at_30s,std_10s_LP_Fp1-C3_at_30s,max-min_10s_LP_Fp1-C3_at_30s,mean_10s_LP_C3-O1_at_30s,min_10s_LP_C3-O1_at_30s,max_10s_LP_C3-O1_at_30s,std_10s_LP_C3-O1_at_30s,max-min_10s_LP_C3-O1_at_30s,mean_10s_RP_Fp2-C4_at_30s,min_10s_RP_Fp2-C4_at_30s,max_10s_RP_Fp2-C4_at_30s,std_10s_RP_Fp2-C4_at_30s,max-min_10s_RP_Fp2-C4_at_30s,mean_10s_RP_C4-O2_at_30s,min_10s_RP_C4-O2_at_30s,max_10s_RP_C4-O2_at_30s,std_10s_RP_C4-O2_at_30s,max-min_10s_RP_C4-O2_at_30s,mean_10s_RR_Fp2-T4_at_30s,min_10s_RR_Fp2-T4_at_30s,max_10s_RR_Fp2-T4_at_30s,std_10s_RR_Fp2-T4_at_30s,max-min_10s_RR_Fp2-T4_at_30s,mean_10s_RR_T4-O2_at_30s,min_10s_RR_T4-O2_at_30s,max_10s_RR_T4-O2_at_30s,std_10s_RR_T4-O2_at_30s,max-min_10s_RR_T4

In [46]:
# join
df_test = df_test.join(df_all_diff_features, on = "eeg_id", how = "left")
df_test.head()

spectrogram_id,eeg_id,patient_id,mean_5s_LL_Fp1-T3_at_20s,min_5s_LL_Fp1-T3_at_20s,max_5s_LL_Fp1-T3_at_20s,std_5s_LL_Fp1-T3_at_20s,max-min_5s_LL_Fp1-T3_at_20s,mean_5s_LL_T3-O1_at_20s,min_5s_LL_T3-O1_at_20s,max_5s_LL_T3-O1_at_20s,std_5s_LL_T3-O1_at_20s,max-min__5s_LL_T3-O1_at_20s,mean_5s_LP_Fp1-C3_at_20s,min_5s_LP_Fp1-C3_at_20s,max_5s_LP_Fp1-C3_at_20s,std_5s_LP_Fp1-C3_at_20s,max-min_5s_LP_Fp1-C3_at_20s,mean_5s_LP_C3-O1_at_20s,min_5s_LP_C3-O1_at_20s,max_5s_LP_C3-O1_at_20s,std_5s_LP_C3-O1_at_20s,max-min_5s_LP_C3-O1_at_20s,mean_5s_RP_Fp2-C4_at_20s,min_5s_RP_Fp2-C4_at_20s,max_5s_RP_Fp2-C4_at_20s,std_5s_RP_Fp2-C4_at_20s,max-min_5s_RP_Fp2-C4_at_20s,mean_5s_RP_C4-O2_at_20s,min_5s_RP_C4-O2_at_20s,max_5s_RP_C4-O2_at_20s,std_5s_RP_C4-O2_at_20s,max-min_5s_RP_C4-O2_at_20s,mean_5s_RR_Fp2-T4_at_20s,min_5s_RR_Fp2-T4_at_20s,max_5s_RR_Fp2-T4_at_20s,std_5s_RR_Fp2-T4_at_20s,max-min_5s_RR_Fp2-T4_at_20s,mean_5s_RR_T4-O2_at_20s,min_5s_RR_T4-O2_at_20s,max_5s_RR_T4-O2_at_20s,std_5s_RR_T4-O2_at_20s,max-min_5s_RR_T4-O2_at_20s,mean_10s_LL_Fp1-T3_at_20s,min_10s_LL_Fp1-T3_at_20s,max_10s_LL_Fp1-T3_at_20s,std_10s_LL_Fp1-T3_at_20s,max-min_10s_LL_Fp1-T3_at_20s,mean_10s_LL_T3-O1_at_20s,min_10s_LL_T3-O1_at_20s,max_10s_LL_T3-O1_at_20s,std_10s_LL_T3-O1_at_20s,max-min__10s_LL_T3-O1_at_20s,mean_10s_LP_Fp1-C3_at_20s,min_10s_LP_Fp1-C3_at_20s,max_10s_LP_Fp1-C3_at_20s,std_10s_LP_Fp1-C3_at_20s,max-min_10s_LP_Fp1-C3_at_20s,mean_10s_LP_C3-O1_at_20s,min_10s_LP_C3-O1_at_20s,max_10s_LP_C3-O1_at_20s,std_10s_LP_C3-O1_at_20s,max-min_10s_LP_C3-O1_at_20s,mean_10s_RP_Fp2-C4_at_20s,min_10s_RP_Fp2-C4_at_20s,max_10s_RP_Fp2-C4_at_20s,std_10s_RP_Fp2-C4_at_20s,max-min_10s_RP_Fp2-C4_at_20s,mean_10s_RP_C4-O2_at_20s,min_10s_RP_C4-O2_at_20s,max_10s_RP_C4-O2_at_20s,std_10s_RP_C4-O2_at_20s,max-min_10s_RP_C4-O2_at_20s,mean_10s_RR_Fp2-T4_at_20s,min_10s_RR_Fp2-T4_at_20s,max_10s_RR_Fp2-T4_at_20s,std_10s_RR_Fp2-T4_at_20s,max-min_10s_RR_Fp2-T4_at_20s,mean_10s_RR_T4-O2_at_20s,min_10s_RR_T4-O2_at_20s,max_10s_RR_T4-O2_at_20s,std_10s_RR_T4-O2_at_20s,max-min_10s_RR_T4-O2_at_20s,mean_5s_LL_Fp1-T3_at_30s,min_5s_LL_Fp1-T3_at_30s,max_5s_LL_Fp1-T3_at_30s,std_5s_LL_Fp1-T3_at_30s,max-min_5s_LL_Fp1-T3_at_30s,mean_5s_LL_T3-O1_at_30s,min_5s_LL_T3-O1_at_30s,max_5s_LL_T3-O1_at_30s,std_5s_LL_T3-O1_at_30s,max-min__5s_LL_T3-O1_at_30s,mean_5s_LP_Fp1-C3_at_30s,min_5s_LP_Fp1-C3_at_30s,max_5s_LP_Fp1-C3_at_30s,std_5s_LP_Fp1-C3_at_30s,max-min_5s_LP_Fp1-C3_at_30s,mean_5s_LP_C3-O1_at_30s,min_5s_LP_C3-O1_at_30s,max_5s_LP_C3-O1_at_30s,std_5s_LP_C3-O1_at_30s,max-min_5s_LP_C3-O1_at_30s,mean_5s_RP_Fp2-C4_at_30s,min_5s_RP_Fp2-C4_at_30s,max_5s_RP_Fp2-C4_at_30s,std_5s_RP_Fp2-C4_at_30s,max-min_5s_RP_Fp2-C4_at_30s,mean_5s_RP_C4-O2_at_30s,min_5s_RP_C4-O2_at_30s,max_5s_RP_C4-O2_at_30s,std_5s_RP_C4-O2_at_30s,max-min_5s_RP_C4-O2_at_30s,mean_5s_RR_Fp2-T4_at_30s,min_5s_RR_Fp2-T4_at_30s,max_5s_RR_Fp2-T4_at_30s,std_5s_RR_Fp2-T4_at_30s,max-min_5s_RR_Fp2-T4_at_30s,mean_5s_RR_T4-O2_at_30s,min_5s_RR_T4-O2_at_30s,max_5s_RR_T4-O2_at_30s,std_5s_RR_T4-O2_at_30s,max-min_5s_RR_T4-O2_at_30s,mean_10s_LL_Fp1-T3_at_30s,min_10s_LL_Fp1-T3_at_30s,max_10s_LL_Fp1-T3_at_30s,std_10s_LL_Fp1-T3_at_30s,max-min_10s_LL_Fp1-T3_at_30s,mean_10s_LL_T3-O1_at_30s,min_10s_LL_T3-O1_at_30s,max_10s_LL_T3-O1_at_30s,std_10s_LL_T3-O1_at_30s,max-min__10s_LL_T3-O1_at_30s,mean_10s_LP_Fp1-C3_at_30s,min_10s_LP_Fp1-C3_at_30s,max_10s_LP_Fp1-C3_at_30s,std_10s_LP_Fp1-C3_at_30s,max-min_10s_LP_Fp1-C3_at_30s,mean_10s_LP_C3-O1_at_30s,min_10s_LP_C3-O1_at_30s,max_10s_LP_C3-O1_at_30s,std_10s_LP_C3-O1_at_30s,max-min_10s_LP_C3-O1_at_30s,mean_10s_RP_Fp2-C4_at_30s,min_10s_RP_Fp2-C4_at_30s,max_10s_RP_Fp2-C4_at_30s,std_10s_RP_Fp2-C4_at_30s,max-min_10s_RP_Fp2-C4_at_30s,mean_10s_RP_C4-O2_at_30s,min_10s_RP_C4-O2_at_30s,max_10s_RP_C4-O2_at_30s,std_10s_RP_C4-O2_at_30s,max-min_10s_RP_C4-O2_at_30s,mean_10s_RR_Fp2-T4_at_30s,min_10s_RR_Fp2-T4_at_30s,max_10s_RR_Fp2-T4_at_30s,std_10s_RR_Fp2-T4_at_30s,max-min_10s_RR_Fp2-T4_at_30s,mean_10s_RR_T4-O2_at_30s,min_10s_RR_T4-O2_at_30s,max_10s_RR_T4-O2_at_30s,std_10s_RR_T4-O

In [47]:
df_test.columns

['spectrogram_id',
 'eeg_id',
 'patient_id',
 'mean_5s_LL_Fp1-T3_at_20s',
 'min_5s_LL_Fp1-T3_at_20s',
 'max_5s_LL_Fp1-T3_at_20s',
 'std_5s_LL_Fp1-T3_at_20s',
 'max-min_5s_LL_Fp1-T3_at_20s',
 'mean_5s_LL_T3-O1_at_20s',
 'min_5s_LL_T3-O1_at_20s',
 'max_5s_LL_T3-O1_at_20s',
 'std_5s_LL_T3-O1_at_20s',
 'max-min__5s_LL_T3-O1_at_20s',
 'mean_5s_LP_Fp1-C3_at_20s',
 'min_5s_LP_Fp1-C3_at_20s',
 'max_5s_LP_Fp1-C3_at_20s',
 'std_5s_LP_Fp1-C3_at_20s',
 'max-min_5s_LP_Fp1-C3_at_20s',
 'mean_5s_LP_C3-O1_at_20s',
 'min_5s_LP_C3-O1_at_20s',
 'max_5s_LP_C3-O1_at_20s',
 'std_5s_LP_C3-O1_at_20s',
 'max-min_5s_LP_C3-O1_at_20s',
 'mean_5s_RP_Fp2-C4_at_20s',
 'min_5s_RP_Fp2-C4_at_20s',
 'max_5s_RP_Fp2-C4_at_20s',
 'std_5s_RP_Fp2-C4_at_20s',
 'max-min_5s_RP_Fp2-C4_at_20s',
 'mean_5s_RP_C4-O2_at_20s',
 'min_5s_RP_C4-O2_at_20s',
 'max_5s_RP_C4-O2_at_20s',
 'std_5s_RP_C4-O2_at_20s',
 'max-min_5s_RP_C4-O2_at_20s',
 'mean_5s_RR_Fp2-T4_at_20s',
 'min_5s_RR_Fp2-T4_at_20s',
 'max_5s_RR_Fp2-T4_at_20s',
 'std_5s_RR_Fp

In [48]:
# Testデータでは、eeg_id, patient_id, spectrogram_id, label_idは1つしかない
FEATURES = df_test.drop(["eeg_id", "spectrogram_id", "patient_id"]).columns
FEATURES

['mean_5s_LL_Fp1-T3_at_20s',
 'min_5s_LL_Fp1-T3_at_20s',
 'max_5s_LL_Fp1-T3_at_20s',
 'std_5s_LL_Fp1-T3_at_20s',
 'max-min_5s_LL_Fp1-T3_at_20s',
 'mean_5s_LL_T3-O1_at_20s',
 'min_5s_LL_T3-O1_at_20s',
 'max_5s_LL_T3-O1_at_20s',
 'std_5s_LL_T3-O1_at_20s',
 'max-min__5s_LL_T3-O1_at_20s',
 'mean_5s_LP_Fp1-C3_at_20s',
 'min_5s_LP_Fp1-C3_at_20s',
 'max_5s_LP_Fp1-C3_at_20s',
 'std_5s_LP_Fp1-C3_at_20s',
 'max-min_5s_LP_Fp1-C3_at_20s',
 'mean_5s_LP_C3-O1_at_20s',
 'min_5s_LP_C3-O1_at_20s',
 'max_5s_LP_C3-O1_at_20s',
 'std_5s_LP_C3-O1_at_20s',
 'max-min_5s_LP_C3-O1_at_20s',
 'mean_5s_RP_Fp2-C4_at_20s',
 'min_5s_RP_Fp2-C4_at_20s',
 'max_5s_RP_Fp2-C4_at_20s',
 'std_5s_RP_Fp2-C4_at_20s',
 'max-min_5s_RP_Fp2-C4_at_20s',
 'mean_5s_RP_C4-O2_at_20s',
 'min_5s_RP_C4-O2_at_20s',
 'max_5s_RP_C4-O2_at_20s',
 'std_5s_RP_C4-O2_at_20s',
 'max-min_5s_RP_C4-O2_at_20s',
 'mean_5s_RR_Fp2-T4_at_20s',
 'min_5s_RR_Fp2-T4_at_20s',
 'max_5s_RR_Fp2-T4_at_20s',
 'std_5s_RR_Fp2-T4_at_20s',
 'max-min_5s_RR_Fp2-T4_at_20s',

In [49]:
# pandasに変換
df_test = df_test.to_pandas()

# Train CatBoost
CatBoost は、超高速トレーニングのために (パラメーター `task_type='GPU'` を追加すると) 両方の Kaggle T4 GPU を自動的に使用する。

In [50]:
import catboost as cat
from catboost import CatBoostClassifier, Pool
print('CatBoost version',cat.__version__)

CatBoost version 1.2.2


## Create Model Folder

In [51]:
%%time
from sklearn.model_selection import KFold, GroupKFold
import json

preds = []

for i in range(5):
    print(i,', ',end='')
    model = CatBoostClassifier(task_type='GPU')
    model.load_model(f'../input/catboost-eeg-feature-ver{VER}-train/models/CAT_v{VER}_f{i}.cat')
    
    test_pool = Pool(
        data = df_test[FEATURES]
    )
    
    pred = model.predict_proba(test_pool)
    preds.append(pred)
pred = np.mean(preds,axis=0)
print()
print('Test preds shape',pred.shape)


0 , 1 , 2 , 3 , 4 , 
Test preds shape (1, 6)
CPU times: total: 0 ns
Wall time: 41.3 ms


In [52]:
sub = pd.DataFrame({'eeg_id':df_test.eeg_id.values})
TARGETS = ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']
sub[TARGETS] = pred
sub.to_csv('submission.csv',index=False)
print('Submissionn shape',sub.shape)
sub.head()

Submissionn shape (1, 7)


,eeg_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,3911565283,0.121312,0.091916,0.047653,0.104651,0.147011,0.487457


In [53]:
# SANITY CHECK TO CONFIRM PREDICTIONS SUM TO ONE
sub.iloc[:,-6:].sum(axis=1)

0    1.0
dtype: float64